# FanDuel data

The purpose of this file is to take in an upcoming FanDuel competition (csv) and generate the inputs necessary for the ML model

In [50]:
import nbimporter
import pandas as pd
from datetime import datetime
import player_matches_data as pm_data
import games_data_OCT as games_data
import pickle
import numpy as np

In [57]:
COMPETITION_PATH = "../data/fan_dual_input.csv"
NAME_ID_MAPPING_PATH = "../data/name_id_mapping.csv"
OLD_DATA_PATH = "../data/20100101_20191129_player_matches.csv"
PLAYER_SEASONS_PATH = "../data/2009_2019_player_seasons_unique.csv"
NB_PLAYERS_PER_TEAM = 7
ROLLING_GAMES_WINDOW = 5

In [52]:
def add_player_id(row):
    name = row['name']
    ids = name_id_mapping[name_id_mapping.name == name]['player_id']
    
    if len(ids) > 0:
        return ids.iloc[0]
    else:
        print("Please update player_id for name " + name)
        return None

In [53]:
def add_home_game(row):
    return row['team_key'] == row['Game'].split("@")[1]

In [62]:
def generate_inputs(competition_path, name_id_mapping_path, old_data_path, player_season_path, nb_players_per_team, rolling_games_window):
    competition = pd.read_csv(competition_path)
    name_id_mapping = pd.read_csv(name_id_mapping_path)
    player_seasons = pd.read_csv(player_season_path)
    old_data = pd.read_csv(old_data_path)
    
    # Adding todays dates for the competition 
    date = datetime.today()
    year = date.year
    month = date.month
    day = date.day

    date = datetime(year, month, day) # strips precision
    competition['date'] = date
    competition['year'] = year
    competition['month'] = month
    competition['day'] = day
    competition = competition.rename(
        columns={
            'Nickname': 'name',
            'Team': 'team_key',
            'Opponent': 'opponent_key'
        }
    )
    competition['home_game'] = competition.apply(add_home_game, axis = 1)
    competition['player_id'] = competition.apply(add_player_id, axis = 1)
    fd_complete = pm_data.add_game_ids(competition)
    fd_complete = pm_data.add_season_start_year(fd_complete)
    stats = ['secs', 'fgm', 'fga', '3pm', '3pa', 'ftm', 'fta', 'orb', 'drb', 'ast', 'stl', 'blk', 'tvr', 'pf', 'fp']
    for stat in stats:
        fd_complete[stat] = 0
    fd_complete = pm_data.roll_with_old_data(fd_complete, old_data, stats, rolling_games_window)
    model_input = games_data.create_games_data(fd_complete, player_seasons, nb_players_per_team, rolling_games_window)

    competition.to_csv("../data/competition.csv", index = False)
    model_input.to_csv("../data/model_input.csv", index = False)
    
    return competition, model_input

In [63]:
competition, model_input = generate_inputs(COMPETITION_PATH, NAME_ID_MAPPING_PATH, OLD_DATA_PATH, PLAYER_SEASONS_PATH, NB_PLAYERS_PER_TEAM, ROLLING_GAMES_WINDOW)